## Install libraries

In [ ]:
! pip install --quiet langchain  langchain_experimental pandas mysql-connector-python gpt4all langchainhub langchain_cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ! pip install --quiet langchain langchain_openai langchain_experimental langchain-anthropic pandas mysql-connector-python gpt4all langchainhub

In [ ]:
# Create the Cohere chat model
from langchain_cohere.chat_models import ChatCohere
from langchain_community.tools.tavily_search import TavilySearchResults
import os
# import openai
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
import sqlite3
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.runnables import ensure_config
import pandas as pd
from gpt4all import Embed4All

import numpy as np
from langchain.agents import AgentType
from langchain.prompts import ChatPromptTemplate
# from langchain.agents import AgentExecutor, create_react_agent
# from langchain_openai import ChatOpenAI






## Create a database and table for sample example tools

In [ ]:
def create_database():
    conn = sqlite3.connect('database.db')

    cursor = conn.cursor()

    with open('database.sql', 'r') as sql_file:
        sql_script = sql_file.read()

    try:
        cursor.executescript(sql_script)
        conn.commit()
        print("SQL script executed successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    cursor.close()
    conn.close()



create_database()

SQL script executed successfully.


In [ ]:
@tool
def search_flights(destination, origin):
    """Search for flights based on destination.
        Values are Persian..
        You have to only search Persian for destination and origin fields"""

    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    params = []
    query = "SELECT info_flight.id, info_flight.day, info_flight.origin, destination.airport_destination, info_flight.date FROM info_flight join destination on id_destination=destination.id  WHERE 1 = 1"

    if destination:
      query += " AND destination.airport_destination = ?"
      params.append('فرودگاه '+ destination)
    if origin:
      query += " AND info_flight.origin = ?"
      params.append(origin)

    cursor.execute(query, params)
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return results



@tool
def cancel_ticket(ticket_id):
    """Cancel the user's ticket and remove it from the database."""

    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute(
        "SELECT * FROM ticket_airplane WHERE ticket_id = ?", (ticket_id,)
    )
    current_flight = cursor.fetchone()
    if not current_flight:
        cursor.close()
        conn.close()
        print("No existing ticket found for the given ticket number.")
    else:
        # Check the signed-in user actually has this ticket
        cursor.execute(
            "SELECT flight_id FROM ticket_airplane WHERE ticket_id = ?",
            (ticket_id,),
        )
        current_ticket = cursor.fetchone()
        if not current_ticket:
            cursor.close()
            conn.close()
            print(f"Not the owner of ticket {ticket_id}")

        # In a real application, you'd likely add additional checks here to enforce business logic,
        # like "does the new departure airport match the current ticket", etc.
        # While it's best to try to be *proactive* in 'type-hinting' policies to the LLM
        # it's inevitably going to get things wrong, so you **also** need to ensure your
        # API enforces valid behavior
        cursor.execute(
            "DELETE FROM ticket_airplane WHERE ticket_id = ?",
            (ticket_id,),
        )
        conn.commit()

        cursor.close()
        conn.close()
        print("Ticket successfully updated to new flight.")



@tool
def take_ticket(flight_id, passenger_id):
    """take a ticket based on flight id and passenger_id.
       Values for search have to be Persian"""

    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()

    query = "SELECT MAX(seat_number) FROM ticket_airplane WHERE flight_id = ?;"
    cursor.execute(query, (flight_id,))
    seat_number = cursor.fetchone()


    if seat_number is None or seat_number[0] is None:
            seat_number = 0
    else:
            seat_number = seat_number[0]

        # Check if the hotel is full (assuming 10 is the maximum room number)
    if seat_number == 10:
            result = "It is full, You can choose another airpalne."
    else:
          insert_query = "INSERT INTO ticket_airplane (passenger_id, flight_id, seat_number) VALUES (?, ?, ?);"
          cursor.execute(insert_query, (passenger_id, flight_id, int(seat_number)+1))

          conn.commit()

          result = "take a ticket successful. Seat number: " + str(seat_number)

    cursor.close()
    conn.close()

    return result



In [ ]:

@tool
def search_hotel(city):
    """Search for hotel based on city. Values are Persian"""
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    query = "SELECT * from hotel WHERE city=? ;"
    cursor.execute(query, (city,))
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return results



@tool
def book_hotel(hotel_id, passenger_id):
    """Book a hotel based on hotel id and passenger_id. Values are Persian"""
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    query = "SELECT MAX(room_number) FROM hotel_book WHERE hotel_id = ?;"
    cursor.execute(query, (hotel_id,))
    room_number = cursor.fetchone()


    if room_number is None or room_number[0] is None:
        room_number = 0
    else:
        room_number = room_number[0]

    # Check if the hotel is full (assuming 10 is the maximum room number)
    if room_number == 10:
        result = "It is full, You can choose another hotel."
    else:
        insert_query = "INSERT INTO hotel_book (passenger_id, hotel_id, room_number) VALUES (?, ?, ?);"
        cursor.execute(insert_query, (passenger_id, hotel_id, int(room_number) + 1))

        conn.commit()

        result = "Booking successful. Room number: " + str(room_number + 1)

    cursor.close()
    conn.close()

    return result


@tool
def cancel_hotel(passenger_id):
    """Cancel the user's hotel and remove it from the database."""

    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()

    cursor.execute(
        "DELETE FROM hotel_book WHERE passenger_id = ?",
        (passenger_id,),
    )
    conn.commit()

    cursor.close()
    conn.close()
    print("It Successfully cancel.")

#
# def rent_car()
#   logic


#   return


#
# def cancel_car()
#   logic


#   return

In [ ]:

df = pd.read_excel('questions_and_answers.xlsx')
doc = df.to_dict('records')


class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: list, oai_client):
        self._arr = np.array(vectors)
        self._docs = docs
        self._client = oai_client

    @classmethod
    def from_docs(cls, docs, oai_client):
        embeddings = oai_client.embed([doc["Question"] for doc in docs])
        vectors = [emb for emb in embeddings]
        return cls(docs, vectors, oai_client)

    def query(self, query: str, k: int = 5) -> list[dict]:
        embed = self._client.embed([query])

        # "@" is just a matrix multiplication in python
        scores = np.array(embed) @ self._arr.T
        scores = scores[0]
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]



        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]


retriever = VectorStoreRetriever.from_docs(doc, Embed4All())
@tool
def lookup_policy(query: str) -> str:
    """Looks up frequently asked questions (FAQ) in a Persian-language document.

    Parameters:
        query (str): The query or question for which the policy is being looked up.

    Returns:
        str: The policy or answer corresponding to the provided query."""
    docs = retriever.query(query, k=1)
    return "\n\n".join([doc["Answer"] for doc in docs])



os.environ['TAVILY_API_KEY'] = ""
internet_search = TavilySearchResults(max_results=1)

In [ ]:

# Set the API key
os.environ['COHERE_API_KEY'] = ""



# Initialize the LLM model
llm_model = ChatCohere(model="command-r-plus", temperature=0.3)
prompt = ChatPromptTemplate.from_template("{input} Just search in Persian-language in SQLite for every response.")


# Define the tools

# Define the tool names
my_tools = [internet_search,
            lookup_policy,
            search_flights,
            take_ticket,
            cancel_ticket,
            cancel_hotel,
            book_hotel,
            search_hotel]
# Create the agent scratchpad
agent_scratchpad = {}

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm_model,
    tools=my_tools,
    prompt=prompt,
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=my_tools,
    verbose=True
)



questions = {
    "input1": "می خوام از اصفهان برم مشهد برای چه ساعت هایی داره؟",
    "input2":"برای پرواز شماره 3 یه بلیط می خوام رزرو کن لطفا" + "passenger_id=2",
    "input3":"آیا ساعت، قیمت و دیگر مشخصات پروازها روی سایت علی‌بابا قطعی است؟",
    "input4": "هتل های شیراز کجاها هستند؟"
    }

for key, value in questions.items():
    # Invoke the agent with each question
    result = agent_executor.invoke({
        "input": value
    })





> Entering new AgentExecutor chain...

I will search for flights from Isfahan to Mashhad and then list the departure times.
{'tool_name': 'search_flights', 'parameters': {'destination': 'مشهد', 'origin': 'اصفهان'}}
[{'id': 398, 'day': 'دو شنبه 14:20', 'origin': 'اصفهان', 'airport_destination': 'فرودگاه مشهد', 'date': '2024-05-13'}, {'id': 403, 'day': 'دو شنبه 15:30', 'origin': 'اصفهان', 'airport_destination': 'فرودگاه مشهد', 'date': '2024-05-13'}, {'id': 417, 'day': 'دو شنبه 19:45', 'origin': 'اصفهان', 'airport_destination': 'فرودگاه مشهد', 'date': '2024-05-13'}, {'id': 418, 'day': 'دو شنبه 20:00', 'origin': 'اصفهان', 'airport_destination': 'فرودگاه مشهد', 'date': '2024-05-13'}, {'id': 425, 'day': 'دو شنبه 22:55', 'origin': 'اصفهان', 'airport_destination': 'فرودگاه مشهد', 'date': '2024-05-13'}, {'id': 456, 'day': 'سه شنبه 14:20', 'origin': 'اصفهان', 'airport_destination': 'فرودگاه مشهد', 'date': '2024-05-14'}, {'id': 478, 'day': 'سه شنبه 19:45', 'origin': 'اصفهان', 'airport_destinati